# Introduction

Author: Jason Jiang

Revised by: Abigayle Hodson, SEA, LBNL

Revision Date: 03/05/2024




Description: WESTNet is a network model and decision-support tool for analyzing the resource consumption, energy usage, and environmental impacts of California's water supply network. The following script reads in a water linkage dataset with a user-specified year scenario and creates a network where each node represents a water utility and each edge represents the flow of water between a source and target utility. Each node in the network is then "balanced" by calculating a volume-weighted average of electricity consumption associated with the treatment and transmission of water. Users can create subgraphs of all the upstream or downstream nodes of a certain facility and visualize the output using a linear graph, network graph, or sankey diagram.


In [210]:
#mount Google Drive- establishes connection between script and Google Drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

#import packages
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import pandas as pd
import numpy as np
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
import plotly.graph_objects as go
import os, getpass, math, json

#specify the path of where code is stored on your machine
root = '/content/gdrive/MyDrive/WESTNet/'

Mounted at /content/gdrive/


# Define WESTNet object class

In [ ]:
class WESTNet(object):
    """WESTNet: network model and decision-support tool for analyzing
    the resource consumption, energy usage, and environmental
    impacts of California's water supply network.
    """
    def __init__(self, file_name, run_name = None):
        #check that file name has correct path and extension
        assert file_name[-4:] == '.csv'
        assert file_name[:len(root)+5] == root + 'data/'
        #initialize WESTNET object and read in csv data
        self.file = file_name
        self.name = file_name[60:-4];
        self.data = pd.read_csv(file_name)
        self.built = False

    def run(self, year, labels, dot=True):
        """Runs entire network assessment - builds the network using
        the input datafile and then "balances" the graph.

        Args:
            year: An integer specifying the scenario year.
            labels: A boolean indicating if volume and energy values
                associated with link should be recorded in resulting
                network and dot file.
            dot: A boolean indicating if a dotfile of the entire
                network should be saved

        Returns:
            Nothing. Writes pdf and dot files, if args are marked true.
        """
        #intitialize parameters
        self.year = year;

        #get data for selected scenario year
        self.d = self.data[self.data['year'] == year]

        #build network
        build_graph(self, labels)

        #get list of source edges
        get_source_facilities(self)

        #balance network
        balance_graph(self)
        self.built = True

        #write dot file, if arg marked as true
        if dot:
            pydot_file = nx.nx_pydot.to_pydot(self.g)
            pydot_file.write_dot(root+'output/links_'+str(self.year)+'.dot')

    def save_energy_csv(self, file_name, dir='output/', index=False):
        """Saves energy consumption as a .csv file in output directory
        """
        return self.energy.to_csv(root+dir+file_name, index=index)

    def create_subgraph(self, utility, filename, up=True, \
        csv=True):
        """Creates a subgraph of utilities upstream or downstream from a
        specified utility from the entire model.

        Args:
            utility: A string of the utility of interest.
            filename: A string of the name of the dot file to be written
            up: A boolean indicating an upstream or downstream graph
            csv: A boolean indicating whether a csv of the resulting subgraph
                should be written (columns: node, kwh/af)

        Returns:
            A dataframe of the node and kwh/af values from weighting the
            subgraph. Writes dot file to the `output` folder.
        """
        #check if the network has been built yet
        assert self.built, "Must run model first."

        #check that data dictionary is in the correct data folder
        assert 'data_dict_utility.json' in os.listdir(root+"data"), "Missing data dictionary."

        #if upstream graph was selected, use parents function to create a nested list of nodes upstream of specified utility
        if up:
            nested = parents(self.d, utility)
        #if downstream graph was selected, use parents function to create a nested list of nodes downstream of specified utility
        else:
            nested = children(self.d, utility)

        #converted nested upstream/downstream utility trees to flat lists and store in attribute sub_nodes
        self.sub_nodes = flatten(nested)
        subgraph = self.g.subgraph(self.sub_nodes)

        colors = ['green', 'aquamarine', 'orange', 'yellow', 'red'];

        #iterate through utilities and assign color to node
        for node in self.completed_nodes:
            val = (((self.completed_nodes[node] - 0) * (4 - 0)) / (4000 - 0))
            val = int(math.floor(val)) if val <=4 else 4;
            color = colorScale(self.completed_nodes[node])
            self.g.add_node(node, style='filled',fillcolor=color)

        #read in data dictionary
        with open(root+'data/data_dict_utility.json') as f:
            data_dict = json.load(f)

        #identify full name of utility and save dot file under that name in the output folder
        utility_name = data_dict["utility"][utility[:-1]]["supplier"]
        filename_full = utility_name + " " + str(self.year) + " " + filename
        save_dot = root + 'output/' + filename_full + '.dot'

        write_dot(subgraph, save_dot)
        print('dot saved: ' + save_dot)

        #create table for subgraph
        rows = [{'node': node, 'kwh/af':self.completed_nodes[node]} \
            for node in subgraph.nodes()]
        self.subgraph_table = pd.DataFrame(rows)

        #create .csv file of table for subgraph, if specified
        if csv:
            self.subgraph_table.to_csv(root + 'output/' + filename + '_energy_table_full.csv', \
                index=False)
            print('csv saved: ' + root + 'output/' + filename + '_energy_table_full.csv')

        return self.subgraph_table[['node', 'kwh/af']]

    def sankey_csv(self, year, utility, filename):
        """Writes a CSV in source-target-value format to use in the sankey
        visualization tool.

        Args:
            year: The year of the data to use
            utility: A string of the end utility
            filename: A string to name the written CSV

        Returns:
            Nothing.
        """
        nodes = {}
        edges = {}
        node_num = 0

        #create a subset of data with only year of interest
        self.d = self.data[self.data['year'] == year]

        #identify upstream nodes of specified utility
        up_nodes = parents(self.d, utility)

        columns = ['source', 'cumulative_volume_af', 'transmission_kwh/af',
                   'treatment_kwh/af','used_vol_af']

        #create a list of edges to be graphed
        paths = graph_edges(up_nodes, down=False)

        #create a list of dicts for source-target pairs (or edges), as well as edge numbers required for the graph.
        edge_dicts = edge_data(self.d, paths)

        print('csv saved: ' + root + 'output/' + filename + '.csv')

        return compile_sankey(self, year, edge_dicts) \
                  .to_csv(root + 'output/' + filename + '.csv', index=False)

# Define functions for running network assessment

In [ ]:
def build_graph(self, labels):
    #create an empty graph structure and empty dictionaries for nodes and edges
    nodes = {};
    edges = {};
    node_num = 0;
    edge_num = 0;
    g = nx.MultiDiGraph();

    #iterate through the rows in the input file data table
    for index, r in self.d.iterrows():
        #pull volume and electricity intensity values for current row and store in dictionary
        d = r[['cumulative_volume_af','transmission_kwh/af','treatment_kwh/af',
            'used_vol_af']].to_dict();

        #add labels, if requested, and edge
        if labels:
            label_str = "cumul. vol: {} \n transmission: {} \n treatment: {}"\
                .format(d['cumulative_volume_af'], \
                        d['transmission_kwh/af'], \
                        d['treatment_kwh/af'])
            g.add_edge(r['source'], r['target'], label=label_str)
        else:
            g.add_edge(r['source'], r['target'])

        #store edge data (source -> target utility combination) in edges dictionary
        d['number'] = edge_num;
        d['source'] = r['source'];
        d['target'] = r['target'];

        edges[(r['source'],r['target'])] = d;
        edge_num +=1;

        #add nodes for source and target utility in nodes dictionary, if not already in node list
        if r['source'] not in nodes.keys():
            nodes[r['source']] = node_num;
            node_num+=1;

        if r['target'] not in nodes.keys():
            nodes[r['target']] = node_num;
            node_num+=1;

    #save graph
    self.g = g
    self.nodes = nodes
    self.edges = edges

    return g

#returns the source utility for a target utility
def in_nodes(self, node):
    return [e[0] for e in self.g.in_edges(node)]

#identifies primary utilities
def get_source_facilities(self):
    primary_nodes = []

    #iterate through nodes (utilities) in input dataframe
    for node in self.nodes:
        #identify the source (upstream) utility
        upstream_nodes =  in_nodes(self, node);

        #if there are no upstream nodes, set as a primary node
        if len(upstream_nodes) == 0:
            #add node name to primary node list
            primary_nodes.append(node);

    self.primary_nodes = primary_nodes;

#check if a node has been balanced yet
def check_all_complete(nodes, completed_nodes):
    status = [n in completed_nodes.keys() for n in nodes];

    return all(status)

#identify nodes that have not yet been balanced
def get_incomplete_node(nodes, completed_nodes):
    for n in nodes:
        if n not in completed_nodes.keys():
            return n

#balance the network by calculating a weighted average of electricity use for each utility
def balance_graph(self):
    debug_file = open("log.txt", "w+")
    debug_file.write("")
    debug_file.close()

    #create a dict with list of nodes and status (start all as 0)
    queue_of_nodes = list(self.g.nodes());
    status = dict.fromkeys(queue_of_nodes, 0);
    completed_nodes = {};
    test =  dict.fromkeys(queue_of_nodes, 0);

    #remove last utility from queue of nodes
    node = queue_of_nodes.pop();

    #iterate through nodes that need to be balanced
    while len(queue_of_nodes) != 0:

        #identify incoming (source) nodes for target node
        ancestors = in_nodes(self, node);

        #check if source node(s) has been balanced
        if check_all_complete(ancestors, completed_nodes):

            #get attribute data from in edges
            edge_names = self.g.in_edges(node);

            #if there are more than zero in edges to be balanced
            if len(edge_names) > 0:

                #initialize total electricity use
                numerator = 0;

                #initialize total water use
                denominator = 0;

                #iterate though in edges
                for e in edge_names:
                    #pull the data for each edge
                    d = self.edges[e];

                    debug_file = open("log.txt", "a")
                    debug_file.write("link: " + e[0] + ", " + e[1] + "\n")
                    debug_vals = "transmission: {} kwh/af; treatment: {} kwh/af; cumulative vol: {} af \n"\
                        .format(d['transmission_kwh/af'], d['treatment_kwh/af'], d['cumulative_volume_af'])
                    debug_file.write(debug_vals)
                    debug_file.close()

                    #sum the electricity use values for each edge
                    numerator += (d['transmission_kwh/af']
                    + d['treatment_kwh/af'] + status[d['source']])* d['cumulative_volume_af'];

                    #sum the volume for each edge
                    denominator += d['cumulative_volume_af'];

                #calculate the weighted average of electricity use
                weighted_average = numerator / denominator;

                #store weighted average electricity use
                completed_nodes[node] = weighted_average;

                #set node status as complete (balanced)
                status[node] = weighted_average;

            else:
                completed_nodes[node] = status[node];

            #advance to next node in queue
            del node
            node = queue_of_nodes.pop();

        #if source node(s) have not been balanced yet, add to queue of nodes
        else:
            queue_of_nodes.append(node);
            node = queue_of_nodes[0];
            del queue_of_nodes[0]
            #node = get_incomplete_node(ancestors, completed_nodes);

    #store weighted average in completed nodes attribute and energy attribute
    self.completed_nodes = completed_nodes;
    data = [{'node':k, 'kwh/af': v} for k, v in completed_nodes.items()]
    self.energy = pd.DataFrame(data)

#sets color scale for graphs
def colorScale(val, vmin=0, vmax=4500, cmap = cm.RdYlGn_r):
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    m = cm.ScalarMappable(norm=norm, cmap=cmap)

    rgb = m.to_rgba(val)
    return mpl.colors.rgb2hex(rgb)

#save pdf and/or svg file to output directory
def save_graph(self, completed_nodes, path = 'output/', pdf=False, svg=True):
    #define file name based on name of utility and year
    filename = '%(name)s_%(year)s' % {
                    'name':self.name,
                    'year': self.year
            };

    #update colors for each node in graph
    colors = ['green', 'aquamarine', 'orange', 'yellow', 'red'];
    for node in completed_nodes:
        val =  (((completed_nodes[node] - 0) * (4 - 0)) / (4000 - 0)) + 0;
        val = int(math.floor(val)) if val <=4 else 4;
        color = colorScale(completed_nodes[node])
        self.g.add_node(node, style='filled',fillcolor=color)

    #write the dot file
    (self.g, root+path+filename+'.dot');

    #save pdf and/or svg file
    if pdf:
        os.system("""
            dot -Tpdf %(path)s%(filename)s.dot -o %(path)s%(filename)s.pdf
            """% {
            'path': path,
            'filename':filename
            });
    if svg:
        os.system("""
            dot -Tsvg %(path)s%(filename)s.dot -o %(path)s%(filename)s.svg
            """% {
            'path': path,
            'filename':filename
            });
        print('svg saved')

def children(data, source):
    """Constructs a nested list of downstream nodes
    from a specified utility.

    Args:
        data: A Pandas DataFrame.
        source: A string specifying the name of the utility.

    Returns:
        A list, nested in a tree-like structure to indicate
        children (downstream nodes). A child is always represented by another
        (possibly nested) list. An example:

        ['1807041PD', ['1807045PD', ['1807045E']],
                      ['1807043PD', ['1807043E']]]

        Or, nothing if in a recursive call and an end utility
        is reached.
    """
    #create dataframe with just specified utility as the source
    rows = data[data.source == source]

    #if no rows with specified utility as source, return nothing
    if rows.shape[0] == 0:
        return

    #create starting node
    down_nodes = [source]

    #iterate through each row in sorted rows
    for i, row in rows.iterrows():
        r = []
        #identify current and next target utility
        target = row.source
        next_target = row.target

        #if there are no downstream utilities of next target utility, append the next target to the tree
        if data[data.source == next_target].shape[0] == 0:
            r.append([next_target])
        #if there are more utilities downstream of next target, call children function again to create nested list
        else:
            r.append(children(data, next_target))

        down_nodes.extend(r)

    return down_nodes

def parents(data, target):
    """
    Constructs a nested list of nodes upstream
    from a specified utility. Follows the same
    structure as the 'children' function.

    Args:
        data: A Pandas DataFrame.
        target: A string specifying the endpoint
            utility.

    Returns:
        A list that is nested in a tree-like structure to indicate
        parents (upstream nodes). The structure of the list mirrors that in the
        'children' function. For example, if the target is '1806011E':

        ['1806011E', ['1806011PD', ['1806011GW', ...] ...] ...]

        Nothing is returned if, in a recursive call, the
        utility has no further sources.
    """
    #extract rows where the specified utility is the target node (upstream facilities)
    rows = data[data.target == target]

    #if there are no upstream facilities, return nothing
    if rows.shape[0] == 0:
        return

    child = [target]
    #iterate through upstream facilities and build tree of upstream facilities
    for i, row in rows.iterrows():
        p = []
        parent = row.target
        next_target = row.source
        if data[data.target == next_target].shape[0] == 0:
            p.append([next_target])
        else:
            p.append(parents(data, next_target))

        child.extend(p)

    return child


def graph_edges(path, down=True) -> list:
    """ Recursively creates a list of edges.

    Args:
        path: A list of children or parents (use the one
            returned from the `children` or `parents` function).
        down: A boolean indicating if the
            edges are to be built up or downstream.

    Returns:
        A list of edges, in tuples, which is usable in
        a NetworkX graph.
    """
    edges = []
    first_node = path[0]
    branches = path[1:]
    seen = set()
    if down:
        for b in branches:
            if type(b) == list:
                edges.append((first_node, b[0]))
            else:
                edges.append((first_node, b))

            if type(b) == list and len(b) == 1:
                edges.append((first_node, b[0]))
            else:
                nested_edges = graph_edges(b, down)
                edges.extend(nested_edges)


    else:
        for b in branches:
            if type(b) == str:
                edges.append((b, first_node))
            else:
                edges.append((b[0], first_node))

            if type(b) == list:
                if len(b) == 1:
                    edges.append((b[0], first_node))
                else:
                    nested_edges = graph_edges(b, down)
                    edges.extend(nested_edges)
            else:
                edges.append((b, first_node))


    edges =  [x for x in edges if not (x in seen or seen.add(x))][::-1]

    return edges

def edge_data(data, paths):
    """Builds a list of edges with data for computing

    Args:
        data: A Pandas DataFrame instance.
        paths: A list of edges (computed in `graph_edges` function).

    Returns:
        A list of dictionaries containing values in the rows for
        source-target pairs (or edges), as well as edge numbers
        required for the graph.
    """
    edge_num = 0
    columns = ['source', 'cumulative_volume_af', 'transmission_kwh/af',
               'treatment_kwh/af','used_vol_af']

    edge_dicts = []
    for i in paths:
        rows = data[(data.source == i[0]) & (data.target == i[1])]

        edge_dict = {}
        for col in columns:
            edge_dict[col] = rows[col].values[0]

        edge_dict['number'] = edge_num

        edge_num += 1

        edge_dicts.append(edge_dict)
        edge_dict['target'] = i[1]

    return edge_dicts

def flatten(container):
    """Gets rid of any nesting in a list.
    Args:
        container: a nested list

    Returns:
        a flattened list, with any nesting removed.
    """
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

def compile_sankey(self, year, edge_data):
    """Creates the sankey csv. This does the bulk of the work for the
    'sankey_csv' function. It finds missing volumes from each of the links
    and adds extra nodes ("Other" or "Other to {some utility}"). Deletes
    any links/edges that don't have water flowing directly to the end node
    (i.e. some edge connected with the end utility).

    Args:
        year: An integer for the year of data to be used. This is
            specified in the 'sankey_csv' function.
        edge_data: All edges in the network, represented as a list of
            dictionaries. Also is computed in the `sankey_csv` function.

    Returns:
        A Pandas dataframe of the source-target-cumulative_volume_af links
        for the sankey diagram.
    """
    sank_dicts = []
    #iterate through source-target pairs
    for v in edge_data:
        sank_dict = {}

        #if the volume passing between utility pair is less than 1 AF, don't include in Sankey csv
        if v['cumulative_volume_af'] <= 1:
            pass
        #otherwise, add to Sankey csv
        else:
            sank_dict['source'] = v['source']
            sank_dict['target'] = v['target']
            sank_dict['value'] = v['cumulative_volume_af']
            sank_dicts.append(sank_dict)

    #convert sankey dict to a dataframe
    df = pd.DataFrame(sank_dicts)
    data = pd.read_csv(self.file)
    data = data[data['year'] == year]

    nodes = []
    #iterate through source utilities
    for i in df['source'].unique():
        sdict = {}

        #check if resource/end or not
        check_resource = data[data['target'] == i].shape
        if check_resource[0] == 0:
            sdict['is resource'] = True
        else:
            sdict['is resource'] = False
        if i[-1:] == 'E':
            sdict['is end'] = True
        else:
            sdict['is end'] = False

        #sum cumulative volume for water entering and leaving the current utility
        outv = df[df['source'] == i]['value'].sum()
        inv = df[df['target'] == i]['value'].sum()

        extra_case = {}
        #if the flow leaving the current utility is less than the flow entering the current utility and it is not an end node, add extra node
        if outv < inv:
            if not sdict['is end']:
                extra_case['value'] = inv - outv
                extra_case['source'] = i
                extra_case['target'] = "Other"
                nodes.append(extra_case)
        #if the flow leaving the current utility is greater than the flow entering the utility and it is not a resource, add extra node
        elif outv > inv:
            if not sdict['is resource']:
                extra_case['value'] = outv - inv
                extra_case['source'] = "Other to " + i
                extra_case['target'] = i
                nodes.append(extra_case)

    #create dataframe for extra utility nodes
    extra_ = pd.concat([df,pd.DataFrame(nodes)]).reset_index(drop=True)
    a = []
    count = 0
    break_count = 0
    while True:
        drop = []
        #iterate through extra utilities and drop unnecessary additions
        for i, r in extra_.iterrows():
            if extra_[extra_['source'] == r['target']].shape[0] == 0:
                if r['target'][-5:] != 'Other' and r['target'][-1:] != 'E':
                    drop.append(r['target'])
                if extra_[extra_['target'] == r['source']].shape[0] == 0:
                    drop.append(r['source'])

            extra_ = extra_.loc[[k for k, v in extra_.iterrows() if v['target'] not in drop]]
            count += len(drop)
            a.extend(drop)
        if count == len(a):
            break_count += 1
        if break_count == 15:
            break
    return extra_

# Run network assessment and create visuals

#### Read in the desired water linkage dataset as a WESTNet object



In [ ]:
w = WESTNet(root+'data/links_021520_15L.csv')

#### Run the entire network assessment
Choose the scenario year and run the assessment. Saves a dot file to the output folder.

In [ ]:
w.run(2015, labels=False, dot=True)

<ipython-input-120-3071a78d218a>:49: DeprecationWarning:

nx.nx_pydot.to_pydot depends on the pydot package, which has known issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723



Visualize the network using a [graphviz tool](https://dreampuf.github.io/GraphvizOnline/).

OR on Jenn's Anaconda Prompt, go to:

cd C:\Users\jenndraut\Google Drive\jason_jiang_westnet\output

For linear graph: dot -Tsvg "links_XXX.dot" > CAntwrk.svg

For network graph: dot -Tsvg "links_XXX.dot" -Kfdp -o CAntwrkWeb.svg

#### Filtering utilities in the network
Filter the nodes upstream or downstream from a specific utility *after* the assessment has completed. The arguments for `.create_subgraph(...)` are:

1. **Utility (required)** - A string of the utility name
2. **Filename (required)** - The name of the dot file that will be saved. Will use the end utility when saving, resulting dot file will be written to `../output`.
3. **Up (default=True)** - True if you want an upstream graph, false if downstream.
4. **csv (default=True)** -  If true, the table of volume weighted averages will be saved to a csv.


In [ ]:
w.create_subgraph("1807307E", "act2015", up=True, csv=True)

dot saved: /content/gdrive/MyDrive/WESTNet/output/San Diego City of 2015 act2015.dot
csv saved: /content/gdrive/MyDrive/WESTNet/output/act2015_energy_table_full.csv


,node,kwh/af
0,SW1803SWP18,1539.196566
1,SW1802SWP01,0.000000
2,SW1809SWP25,6202.673950
3,SFBayDelta,0.000000
4,Lk_Havasu,0.000000
5,SW1809SWP20,5331.880429
6,SW1809SWP21,5331.880429
7,1807307E,2434.023570
8,1807307NPD,344.000000
9,SW1807SWP22,5680.743138


Create a Sankey diagram to visualize flow for the selected facility and year

In [ ]:
filename = 'sanfrancisco2015'
w.sankey_csv(2020, "1803035E", filename)
sankey_data = pd.read_csv(root + 'output/' + filename + '.csv')

#assign a number to each utility
labels = pd.DataFrame()
labels['utility'] = pd.concat([sankey_data['source'],sankey_data['target']]).unique()
labels['number'] = range(0,labels['utility'].shape[0])

for row in range(sankey_data.shape[0]):
  source = sankey_data.at[row, 'source']
  target = sankey_data.at[row, 'target']
  sankey_data.at[row, 'source'] = labels.loc[labels['utility'] == source, 'number'].values[0]
  sankey_data.at[row, 'target'] = labels.loc[labels['utility'] == target, 'number'].values[0]

#create a Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(labels['utility']),
    ),
    link = dict(
      source = list(sankey_data['source']),
      target = list(sankey_data['target']),
      value = list(sankey_data['value'])
  ))])

fig.show()

csv saved: /content/gdrive/MyDrive/WESTNet/output/sanfrancisco2015.csv
